# Assignmnet 3

## Question 1 

###  Importing modules

In [1]:
import numpy as np
import vtk
from vtk.util.numpy_support import numpy_to_vtk, vtk_to_numpy

In [2]:
def load_dataset(path):
    reader = vtk.vtkXMLImageDataReader()
    reader.SetFileName(path)
    reader.Update()
    data = reader.GetOutput()
    return data


### Loading Data

In [3]:
data = load_dataset("tornado3d_vector.vti")
# print(data)

In [4]:
bounds = data.GetBounds()

# print(bounds)

### Accessing Vectors array from given Data

In [5]:

def get_vector_at_point(data, point):
    # Create a probe filter
    probe = vtk.vtkProbeFilter()
    probe.SetSourceData(data)
    
    # Set up points for probing
    points = vtk.vtkPoints()
    points.InsertNextPoint(point)
    
    # Create polydata and set points
    polydata = vtk.vtkPolyData()
    polydata.SetPoints(points)
    
    # Connect polydata to the probe filter
    probe.SetInputData(polydata)
    probe.Update()
    
    # Retrieve probed data
    probed_data = probe.GetOutput()
    
    # Get vectors from probed data
    vectors = vtk_to_numpy(probed_data.GetPointData().GetVectors())
    
    # Check if vectors are available
    if vectors is None:
        return np.array([0.0, 0.0, 0.0])  # No vectors found, return zero vector
    else:
        return vectors[0]  # Return the vector at the queried point

    


### RK4 Integration 

In [6]:


def rk4_integration(data, seed, step_size, max_steps):
    # Initialize streamline with seed point
    streamline = [seed]
    current_point = seed
    
    for _ in range(max_steps):
        # Get vector at current point
        vector = get_vector_at_point(data, current_point)
        # print("Hi")
        # print(vector)
        
        # Runge-Kutta 4 integration
        k1 = step_size * vector
        k2 = step_size * get_vector_at_point(data, current_point + 0.5 * k1)
        k3 = step_size * get_vector_at_point(data, current_point + 0.5 * k2)
        k4 = step_size * get_vector_at_point(data, current_point + k3)
        
        next_point = current_point + (k1 + 2 * k2 + 2 * k3 + k4) / 6
        
        # Check if next point is within bounds
        bounds = data.GetBounds()
        if (next_point[0] <= bounds[0] or next_point[0] >= bounds[1] or
            next_point[1] <= bounds[2] or next_point[1] >= bounds[3] or
            next_point[2] <= bounds[4] or next_point[2] >= bounds[5]):
            break
        
        # Add next point to streamline
        streamline.append(next_point.tolist())
        current_point = next_point
    
    return np.array(streamline)


### Visualizing Streamlines 

In [7]:
def visualize_streamline(streamline_points):
    # Create a polydata to hold the streamline points
    poly_data = vtk.vtkPolyData()
    points = vtk.vtkPoints()
    for point in streamline_points:
        points.InsertNextPoint(point)
    poly_data.SetPoints(points)

    # Create lines to connect the points
    lines = vtk.vtkCellArray()
    for i in range(len(streamline_points) - 1):
        line = vtk.vtkLine()
        line.GetPointIds().SetId(0, i)
        line.GetPointIds().SetId(1, i + 1)
        lines.InsertNextCell(line)

    poly_data.SetLines(lines)

    # Write the polydata to a VTKPolyData file
    writer = vtk.vtkXMLPolyDataWriter()
    writer.SetFileName("streamline.vtp")
    writer.SetInputData(poly_data)
    writer.Write()
    
    # Create renderer and render window
    renderer = vtk.vtkRenderer()
    render_window = vtk.vtkRenderWindow()
    render_window.AddRenderer(renderer)
    
    # Create mapper and actor
    mapper = vtk.vtkPolyDataMapper()
    mapper.SetInputData(poly_data)
    actor = vtk.vtkActor()
    actor.SetMapper(mapper)
    actor.GetProperty().SetColor(84/255, 176/255, 59/255)  # Set color to green
    
    # Add actor to the renderer
    renderer.AddActor(actor)
    
    # Set background color
    renderer.SetBackground(1, 1, 1)  # White background
    
    # Create render window interactor
    render_window_interactor = vtk.vtkRenderWindowInteractor()
    render_window_interactor.SetRenderWindow(render_window)
    
    # Set up interactor style
    style = vtk.vtkInteractorStyleTrackballCamera()
    render_window_interactor.SetInteractorStyle(style)
    
    # Start the rendering loop
    render_window.Render()
    render_window_interactor.Start()


### Taking seed location as user input  and calculating Streamlines points

In [8]:
def main():
    # Get user input for seed location
    seed = [float(input("Enter x coordinate of seed point: ")),
            float(input("Enter y coordinate of seed point: ")),
            float(input("Enter z coordinate of seed point: "))]

    # seed = [9,9,9]

    # Define parameters
    step_size = 0.05
    max_steps = 1000

    # Perform RK4 integration in both directions
    forward_streamline = rk4_integration(data, seed, step_size, max_steps)
    backward_streamline = rk4_integration(data, seed, -step_size, max_steps)
    backward_streamline = backward_streamline[1:]

    # Combine forward and backward streamlines
    combined_streamline = np.concatenate((np.flip(backward_streamline, axis=0), forward_streamline))

    # Save combined streamline as VTKPolyData file
    visualize_streamline(combined_streamline)
    # save_streamline_as_vtp(combined_streamline, "streamline.vtp")
    print("vtp file generated successfully.")
    # print(len(combined_streamline))
    

if __name__ == "__main__":
    main()

Enter x coordinate of seed point:  0
Enter y coordinate of seed point:  0
Enter z coordinate of seed point:  7


vtp file generated successfully.


In [9]:
# print(len(combined_streamline))